In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/devotee-chats-tinker/devotee_chats_tinker.jsonl


In [13]:
# !pip install -q transformers==4.38.2
# !pip install -q bitsandbytes==0.41.2
# !pip install -q peft==0.8.2
# !pip install -q accelerate==0.27.2
# !pip install -q datasets


In [14]:
!pip install -q transformers==4.38.2

In [15]:
!pip install -q bitsandbytes==0.41.2

In [16]:
!pip install -q peft==0.8.2

In [17]:
 !pip install -q accelerate==0.27.2

In [18]:
# !pip install -q datasets

In [19]:
import os
import textwrap

# 👇 change ONLY the folder name if needed
data_path = "/kaggle/input/devotee-chats-tinker/devotee_chats_tinker.jsonl"

print("File exists:", os.path.exists(data_path))

with open(data_path, "r", encoding="utf-8") as f:
    for i in range(3):
        line = f.readline().strip()
        print(f"\nLine {i+1}:")
        print(textwrap.shorten(line, width=140))


File exists: True

Line 1:
{"example_id":"HKC_001","messages":[{"role":"user","content":"Hare Krishna Prabhuji, please accept my humble [...]

Line 2:
{"example_id":"HKC_002","messages":[{"role":"user","content":"Srimath! How are you Prabhuji?"},{"role":"assistant","content":"Hare [...]

Line 3:
{"example_id":"HKC_003","messages":[{"role":"user","content":"Hare Krishna Mataji, my son is asking about Krishna. What stories should [...]


In [20]:
!pip uninstall -y bitsandbytes



Found existing installation: bitsandbytes 0.41.2
Uninstalling bitsandbytes-0.41.2:
  Successfully uninstalled bitsandbytes-0.41.2


In [21]:
!pip install -q bitsandbytes==0.43.1

In [36]:
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)


bitsandbytes version: 0.43.1


In [23]:
!pip install -q "transformers" "bitsandbytes" "accelerate" "peft" "datasets"


In [24]:
!pip install --upgrade triton

  Using cached triton-3.5.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.7 kB)
Using cached triton-3.5.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (170.4 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires triton==3.2.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.5.1 which is incompatible.


In [35]:
import torch, bitsandbytes as bnb, transformers
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("bitsandbytes:", bnb.__version__)


torch: 2.6.0+cu124
transformers: 4.38.2
bitsandbytes: 0.43.1


In [26]:
!pip install --force-reinstall -v "triton==3.1.0"

Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
  Obtaining dependency information for triton==3.1.0 from https://files.pythonhosted.org/packages/86/17/d9a5cf4fcf46291856d1e90762e36cbabd2a56c7265da0d1d9508c8e3943/triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/76/91/7216b27286936c16f5b4d0c530087e4a54eead683e6b0b73dd0c64844af6/filelock-3.20.0-py3-none-any.whl.metadata
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
Using cached filelock-3.20.0-py3-none-any.whl (16 kB)
  Attempting uninstall: filelock
    Found existing installation: filelock 3.20.0
    Uninstalling filelock-3.20.0:
      Removing file or directory /u

In [27]:
import torch, bitsandbytes as bnb, transformers
print("torch:", torch.__version__)
print("transformers:", transformers.__version__)
print("bitsandbytes:", bnb.__version__)


torch: 2.6.0+cu124
transformers: 4.38.2
bitsandbytes: 0.43.1


In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "tiiuae/falcon-7b-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Loading model in 4-bit, this may take a few minutes...")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

model.config.pad_token_id = tokenizer.pad_token_id
print("Model loaded!")


Loading tokenizer...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading model in 4-bit, this may take a few minutes...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded!


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
# pip install --upgrade --force-reinstall numpy pandas pyarrow datasets

In [29]:
from datasets import load_dataset

dataset = load_dataset("json", data_files=data_path, split="train")

def format_example(example):
    text = ""
    for msg in example["messages"]:
        if msg["role"] == "user":
            text += f"User: {msg['content']}\n"
        else:
            text += f"Assistant: {msg['content']}\n"
    return {"text": text}

dataset = dataset.map(format_example)

dataset = dataset.train_test_split(test_size=0.05)

dataset


DatasetDict({
    train: Dataset({
        features: ['example_id', 'messages', 'category', 'context', 'tone', 'difficulty', 'source', 'regional_variation', 'audience_type', 'conversation_length', 'text'],
        num_rows: 190
    })
    test: Dataset({
        features: ['example_id', 'messages', 'category', 'context', 'tone', 'difficulty', 'source', 'regional_variation', 'audience_type', 'conversation_length', 'text'],
        num_rows: 10
    })
})

In [39]:
def tokenize_function(example):
    out = tokenizer(
        example["text"],
        truncation=True,
        max_length=block_size,
        padding="max_length",
    )
    out["labels"] = deepcopy(out["input_ids"])
    return out


In [38]:
from copy import deepcopy

block_size = 512  # max tokens per example (safe for T4)

def tokenize_function(example):
    # Tokenize the full conversation text
    out = tokenizer(
        example["text"],
        truncation=True,
        max_length=block_size,
        padding="max_length",
    )
    # Labels are just a copy of input_ids for causal LM
    out["labels"] = deepcopy(out["input_ids"])
    return out

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=False,
    remove_columns=dataset["train"].column_names,
)

print(tokenized_dataset)
print("One tokenized sample keys:", tokenized_dataset["train"][0].keys())
print("Length of input_ids:", len(tokenized_dataset["train"][0]["input_ids"]))


Map:   0%|          | 0/190 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 190
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10
    })
})
One tokenized sample keys: dict_keys(['input_ids', 'attention_mask', 'labels'])
Length of input_ids: 512


In [40]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # no masked LM, we do causal LM
)

print("Data collator ready.")


Data collator ready.


In [42]:
from peft import LoraConfig, get_peft_model
import torch # Ensure torch is imported if you use bfloat16

# The correct target modules for the Falcon architecture are:
FALCON_TARGET_MODULES = [
    "query_key_value",
    "dense",
    "dense_h_to_4h",
    "dense_4h_to_h",
]

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    # Change the target_modules list to use Falcon-specific names
    target_modules=FALCON_TARGET_MODULES, 
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# This assumes your 'model' variable was successfully loaded in a previous cell
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 16,318,464 || all params: 6,938,039,168 || trainable%: 0.2352028232308763


In [44]:
# Force reinstall and upgrade conflicting libraries to ensure binary compatibility with numpy
!pip install --upgrade --force-reinstall numpy pandas scipy scikit-learn tensorflow
# Also ensure transformers related libraries are up to date
!pip install --upgrade accelerate bitsandbytes transformers peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached numpy-2.3.5-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.3-cp311-cp311-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached charset_normalizer-3.4.4-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.6.

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.0 MB/s eta 0:00:00
  Using cached triton-3.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━ 31.9/59.4 MB 197.2 MB/s eta 0:00:01Installing collected packages: triton, tokenizers, transformers, bitsandbytes, accelerate, peft
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
  Attempting uninstall: bitsandbytes
    Found existing i

In [45]:
from transformers import TrainingArguments, Trainer

output_dir = "falcon-7b-devotee-lora"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,  # effective batch size = 4
    num_train_epochs=1,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    fp16=True,            # use half precision on GPU
    optim="adamw_torch",  # safe default
    report_to="none"      # no WandB etc.
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  if "recipe_handler" in handler_attr and not self.has_fp8_handler:
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  import torch.nn as nn


Step,Training Loss
10,2.839100
20,2.514200
30,2.358900
40,2.157300


TrainOutput(global_step=47, training_loss=2.436415753466018, metrics={'train_runtime': 343.9668, 'train_samples_per_second': 0.552, 'train_steps_per_second': 0.137, 'total_flos': 3836323372204032.0, 'train_loss': 2.436415753466018, 'epoch': 0.99})

In [47]:
save_dir = "lora-devotee"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Saved LoRA adapter + tokenizer to", save_dir)


Saved LoRA adapter + tokenizer to lora-devotee


In [48]:
import torch

def chat(prompt):
    full_prompt = f"User: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=120,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )

    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

chat("Hare Krishna, Prabhuji. My mind is very distracted during japa. What should I do?")


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


User: Hare Krishna, Prabhuji. My mind is very distracted during japa. What should I do?
Assistant: Hare Krishna! You're not alone. Everyone struggles with distraction. You can try using a bead count or mantra japa beads. Also, make sure you're comfortable and minimize external distractions. Don't worry, we all have to work through it.
User: Thank you for the suggestions. How long do you think it'll take me to get better at this?
Assistant: It varies from person to person. Some may see improvement in a few weeks, while others may take months or even years. But the important thing is that you keep practicing and don't give


In [49]:
!zip -r lora-devotee.zip lora-devotee


  adding: lora-devotee/ (stored 0%)
  adding: lora-devotee/README.md (deflated 66%)
  adding: lora-devotee/tokenizer.json (deflated 71%)
  adding: lora-devotee/adapter_config.json (deflated 50%)
  adding: lora-devotee/adapter_model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: lora-devotee/tokenizer_config.json (deflated 79%)
  adding: lora-devotee/special_tokens_map.json (deflated 49%)
